In [1]:
!pip install --upgrade pip
!pip install loguru opencv-python-headless tabulate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 9.0 MB/s eta 0:00:00:00:0100:01


In [2]:
import torch
import torch.fx
import torch.nn as nn
from torch_tensorrt.fx.utils import LowerPrecision
import torch_tensorrt.fx.tracer.acc_tracer.acc_tracer as acc_tracer
from torch_tensorrt.fx import InputTensorSpec, TRTInterpreter, TRTModule
from torch_tensorrt.fx.tools.trt_splitter import TRTSplitter

In [6]:
model = torch.hub.load("Megvii-BaseDetection/YOLOX", "yolox_nano").cuda().eval()

Using cache found in /root/.cache/torch/hub/Megvii-BaseDetection_YOLOX_main


In [10]:
model.backbone

YOLOPAFPN(
  (backbone): CSPDarknet(
    (stem): Focus(
      (conv): BaseConv(
        (conv): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    )
    (dark2): Sequential(
      (0): DWConv(
        (dconv): BaseConv(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (pconv): BaseConv(
          (conv): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (1): CSPLayer(
        (conv1): BaseConv(
          (conv): Conv2d(32, 16, kernel_size=(1, 1), str

In [7]:
inputs = [torch.randn((1, 3, 480, 640), device=torch.device("cuda"))]

In [11]:
# acc_tracer is a custom fx tracer that maps nodes whose targets are PyTorch operators
# to acc ops.
traced = acc_tracer.trace(model.backbone, inputs)

/root/.pyenv/versions/3.10.12/lib/python3.10/site-packages/torch/fx/operator_schemas.py:194: UserWarning: We were not able to successfully create type hint from the type (<class 'ellipsis'>, slice(<class 'NoneType'>, <class 'NoneType'>, <class 'int'>), slice(<class 'NoneType'>, <class 'NoneType'>, <class 'int'>))
  warnings.warn(f"We were not able to successfully create type hint from the type {x}")
/root/.pyenv/versions/3.10.12/lib/python3.10/site-packages/torch/fx/operator_schemas.py:194: UserWarning: We were not able to successfully create type hint from the type (<class 'ellipsis'>, slice(<class 'NoneType'>, <class 'NoneType'>, <class 'int'>), slice(<class 'int'>, <class 'NoneType'>, <class 'int'>))
  warnings.warn(f"We were not able to successfully create type hint from the type {x}")
/root/.pyenv/versions/3.10.12/lib/python3.10/site-packages/torch/fx/operator_schemas.py:194: UserWarning: We were not able to successfully create type hint from the type (<class 'ellipsis'>, slice(<c

In [12]:
traced

GraphModule(
  (backbone): Module(
    (stem): Module(
      (conv): Module(
        (conv): Module()
        (bn): Module()
      )
    )
    (dark2): Module(
      (0): Module(
        (dconv): Module(
          (conv): Module()
          (bn): Module()
        )
        (pconv): Module(
          (conv): Module()
          (bn): Module()
        )
      )
      (1): Module(
        (conv1): Module(
          (conv): Module()
          (bn): Module()
        )
        (conv2): Module(
          (conv): Module()
          (bn): Module()
        )
        (m): Module(
          (0): Module(
            (conv1): Module(
              (conv): Module()
              (bn): Module()
            )
            (conv2): Module(
              (dconv): Module(
                (conv): Module()
                (bn): Module()
              )
              (pconv): Module(
                (conv): Module()
                (bn): Module()
              )
            )
          )
        )
        (con

In [13]:
# Splitter will split the model into several submodules. The name of submodules will
# be either `run_on_acc_{}` or `run_on_gpu_{}`. Submodules named `run_on_acc_{}` can
# be fully lowered to TensorRT via fx2trt while submodules named `run_on_gpu_{}` has
# unsupported ops and can't be lowered by fx2trt. We can still run `run_on_gpu_{}`
# submodules on Gpu if ops there have cuda implementation, the naming is a bit
# confusing and we'll improve it.
splitter = TRTSplitter(traced, inputs)
splitter

In [15]:
dir(splitter)

['PCIe_BW',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_draw_graph_based_on_node_support',
 '_find_culprit',
 '_lower_model_to_backend',
 '_node_submodule_map',
 'acc_nodes',
 'deps',
 'extend_acc_subgraph',
 'find_deps',
 'find_parent_nodes_of_subgraph',
 'find_reverse_deps',
 'fusions',
 'generate_split_results',
 'get_node_submodule_map',
 'module',
 'node_support_preview',
 'non_acc_submodule_name',
 'operator_support',
 'put_nodes_into_subgraphs',
 'remove_small_acc_subgraphs',
 'sample_input',
 'settings',
 'split',
 'split_preview',
 'starter_nodes',
 'tag',
 'update_deps_for_fusions',
 'update_reverse_deps_for_fusions']

In [17]:
splitter.non_acc_submodule_name

'_run_on_gpu_'

In [18]:
# Preview functionality allows us to see what are the supported ops and unsupported
# ops. We can optionally the dot graph which will color supported ops and unsupported
# ops differently.
splitter.node_support_preview(dump_graph=False)



Supported node types in the model:
acc_ops.getitem: ((), {'input': torch.float32})
acc_ops.cat: ((), {})
acc_ops.conv2d: ((), {'input': torch.float32, 'weight': torch.float32})
acc_ops.batch_norm: ((), {'input': torch.float32, 'running_mean': torch.float32, 'running_var': torch.float32, 'weight': torch.float32, 'bias': torch.float32})
acc_ops.sigmoid: ((), {'input': torch.float32})
acc_ops.mul: ((), {'input': torch.float32, 'other': torch.float32})
acc_ops.add: ((), {'input': torch.float32, 'other': torch.float32})
acc_ops.max_pool2d: ((), {'input': torch.float32})
acc_ops.interpolate: ((), {'input': torch.float32})

Unsupported node types in the model:



"\nSupported node types in the model:\nacc_ops.getitem: ((), {'input': torch.float32})\nacc_ops.cat: ((), {})\nacc_ops.conv2d: ((), {'input': torch.float32, 'weight': torch.float32})\nacc_ops.batch_norm: ((), {'input': torch.float32, 'running_mean': torch.float32, 'running_var': torch.float32, 'weight': torch.float32, 'bias': torch.float32})\nacc_ops.sigmoid: ((), {'input': torch.float32})\nacc_ops.mul: ((), {'input': torch.float32, 'other': torch.float32})\nacc_ops.add: ((), {'input': torch.float32, 'other': torch.float32})\nacc_ops.max_pool2d: ((), {'input': torch.float32})\nacc_ops.interpolate: ((), {'input': torch.float32})\n\nUnsupported node types in the model:\n"

In [19]:
# Split.
split_mod = splitter()

Got 1 acc subgraphs and 0 non-acc subgraphs


In [20]:
# After split we have three submodules, _run_on_acc_0 and _run_on_gpu_1.
print(split_mod.graph)


graph():
    %input_1 : [#users=1] = placeholder[target=input_1]
    %_run_on_acc_0 : [#users=3] = call_module[target=_run_on_acc_0](args = (%input_1,), kwargs = {})
    %getitem : [#users=1] = call_function[target=operator.getitem](args = (%_run_on_acc_0, 0), kwargs = {})
    %getitem_1 : [#users=1] = call_function[target=operator.getitem](args = (%_run_on_acc_0, 1), kwargs = {})
    %getitem_2 : [#users=1] = call_function[target=operator.getitem](args = (%_run_on_acc_0, 2), kwargs = {})
    return (getitem, getitem_1, getitem_2)


In [21]:

def get_submod_inputs(mod, submod, inputs):
    acc_inputs = None

    def get_input(self, inputs):
        nonlocal acc_inputs
        acc_inputs = inputs

    handle = submod.register_forward_pre_hook(get_input)
    mod(*inputs)
    handle.remove()
    return acc_inputs


# Since the model is splitted into three segments. We need to lower each TRT eligible segment.
# If we know the model can be fully lowered, we can skip the splitter part.
for name, _ in split_mod.named_children():
    if "_run_on_acc" in name:
        submod = getattr(split_mod, name)
        # Get submodule inputs for fx2trt
        acc_inputs = get_submod_inputs(split_mod, submod, inputs)

        # fx2trt replacement
        interp = TRTInterpreter(
            submod,
            InputTensorSpec.from_tensors(acc_inputs),
            explicit_batch_dimension=True,
        )
        r = interp.run(lower_precision=LowerPrecision.FP32)
        trt_mod = TRTModule(*r)
        setattr(split_mod, name, trt_mod)


[06/09/2023-23:48:01] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


INFO:torch_tensorrt.fx.fx2trt:TRT INetwork construction elapsed time: 0:00:05.171293
INFO:torch_tensorrt.fx.fx2trt:Build TRT engine elapsed time: 0:01:12.171480


[06/09/2023-23:49:18] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


In [22]:

lowered_model_output = split_mod(*inputs)

# Save and load model
torch.save(split_mod, "trt.pt")
reload_trt_mod = torch.load("trt.pt")
reload_model_output = reload_trt_mod(*inputs)


[06/09/2023-23:49:47] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading


In [25]:
reload_model_output

(tensor([[[[-8.9316e-02, -2.2630e-01,  1.5959e-01,  ...,  6.4901e-01,
             4.1133e-01,  2.4447e-01],
           [-9.0794e-02, -2.5189e-01,  2.3452e-02,  ...,  2.8365e-01,
            -7.1280e-02,  3.4402e-02],
           [-1.4922e-01, -1.5998e-01,  2.4949e-01,  ...,  1.0765e-01,
            -5.0098e-02,  3.9327e-01],
           ...,
           [ 6.6856e-02, -1.3786e-01,  3.6274e-01,  ..., -8.5888e-02,
            -2.2017e-01, -2.4735e-01],
           [-8.3339e-02, -1.9668e-01,  4.3444e-02,  ..., -1.5990e-01,
            -1.9910e-01, -2.7598e-01],
           [ 3.8716e-02, -1.9877e-01,  1.0792e-02,  ..., -2.2676e-02,
            -1.3005e-01, -2.6546e-01]],
 
          [[-8.5041e-02,  4.3165e-01,  6.0169e-01,  ...,  4.6086e-01,
             3.0016e-01, -9.7433e-02],
           [ 3.4876e-01,  1.3553e+00,  1.0851e+00,  ...,  7.7728e-01,
             4.9340e-01, -1.8551e-01],
           [ 1.0991e-01,  3.2507e-01,  8.4735e-02,  ...,  4.2321e-01,
            -7.3836e-02, -2.7390e-01],


In [26]:
# Make sure the results match
regular_model_output = model.backbone(*inputs)

In [27]:
regular_model_output

(tensor([[[[-7.5837e-02, -2.3319e-01,  1.5376e-01,  ...,  6.4456e-01,
             4.0561e-01,  2.5368e-01],
           [-9.1250e-02, -2.5452e-01,  2.2328e-02,  ...,  2.6407e-01,
            -8.5649e-02,  3.2721e-02],
           [-1.5058e-01, -1.6837e-01,  2.4238e-01,  ...,  9.2854e-02,
            -6.9895e-02,  4.0667e-01],
           ...,
           [ 5.4107e-02, -1.6037e-01,  3.2738e-01,  ..., -5.3661e-03,
            -1.1580e-01, -2.6990e-01],
           [-8.1421e-02, -2.0240e-01,  3.9247e-02,  ..., -1.5974e-01,
            -1.5032e-01, -2.4295e-01],
           [ 1.8059e-02, -2.0038e-01, -5.4908e-03,  ..., -5.1034e-02,
            -4.5448e-02, -1.9630e-01]],
 
          [[-9.4694e-02,  4.6168e-01,  5.9875e-01,  ...,  4.6843e-01,
             2.9579e-01, -1.0078e-01],
           [ 3.7546e-01,  1.4308e+00,  1.0985e+00,  ...,  8.1557e-01,
             5.1170e-01, -1.8474e-01],
           [ 1.1958e-01,  3.5246e-01,  1.0415e-01,  ...,  4.5882e-01,
            -5.7397e-02, -2.7289e-01],
